# Estimación del Precio de Venta en Vehículos de Ocasión 🚗

En este notebook se desarrolla un modelo de red neuronal para predecir el precio de venta de vehículos usados a partir de 7 variables independientes. Se incluyen pasos de:
- **Carga y exploración del dataset**
- **Limpieza y preprocesamiento de datos**
- **Ingeniería de características**
- **División en conjuntos de entrenamiento y prueba**
- **Construcción, entrenamiento y evaluación de la red neuronal**

El dataset se asume en un archivo `true_car_listings.csv` con las siguientes columnas:
- **Price**: Variable objetivo.
- **Year**: Año en que se compró el vehículo.
- **Mileage**: Número de kilómetros recorridos.
- **City**: Ciudad en la que se vendió.
- **State**: Estado en el que se vendió.
- **Vin**: Identificador único del vehículo.
- **Make**: Fabricante.
- **Model**: Modelo del vehículo.

#### 📚 Importacion de librerias

In [3]:
# Importamos las librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, LabelEncoder
# Configuración de Matplotlib para gráficos en línea
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
# Configuración de Matplotlib para gráficos en línea
%matplotlib inline

#Importamos modelos predictivos regresion
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.svm import SVR

## 1. Carga y Exploración del Dataset

En este paso se carga el archivo CSV y se exploran las primeras filas y la estructura del dataset para entender mejor los datos con los que trabajaremos.

In [4]:
# Cargamos el dataset
df = pd.read_csv('true_car_listings.csv')
print("Primeras filas del dataset:")
print(df.head())

# Información general del dataset
print("\nInformación del dataset:")
print(df.info())

Primeras filas del dataset:
   Price  Year  Mileage              City State                Vin   Make  \
0   8995  2014    35725           El Paso    TX  19VDE2E53EE000083  Acura   
1  10888  2013    19606  Long Island City    NY  19VDE1F52DE012636  Acura   
2   8995  2013    48851           El Paso    TX  19VDE2E52DE000025  Acura   
3  10999  2014    39922           Windsor    CO  19VDE1F71EE003817  Acura   
4  14799  2016    22142            Lindon    UT  19UDE2F32GA001284  Acura   

          Model  
0    ILX6-Speed  
1    ILX5-Speed  
2    ILX6-Speed  
3    ILX5-Speed  
4  ILXAutomatic  

Información del dataset:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 852122 entries, 0 to 852121
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   Price    852122 non-null  int64 
 1   Year     852122 non-null  int64 
 2   Mileage  852122 non-null  int64 
 3   City     852122 non-null  object
 4   State    852122 non-null  object

## 2. Limpieza y Preprocesamiento de los Datos

### 2.1 Eliminación de Columnas Innecesarias
Eliminamos la columna `Vin` ya que es un identificador único que no aporta información para la predicción.

### 2.2 Gestión de Valores Nulos
Verificamos si existen valores nulos en el dataset y, en este ejemplo, optamos por eliminar las filas que los contengan.

### 2.3 Ingeniería de Características: One-Hot Encoding
Convertimos las variables categóricas (`City`, `State`, `Make` y `Model`) en variables dummy para poder trabajar con ellas en la red neuronal.

### 2.4 Escalado de Variables Numéricas
Las variables numéricas `Year` y `Mileage` se escalan utilizando `StandardScaler` para facilitar el entrenamiento.

In [5]:
# 2.1 Eliminación de la columna 'Vin'
df = df.drop('Vin', axis=1)
print(df.head())

# 2.2 Gestión de valores nulos
print("\nValores nulos por columna:")
print(df.isnull().sum())

# Eliminamos las filas con valores nulos
df = df.dropna()

# 2.3 Label Encoding para las variables categóricas
categorical_cols = ['City', 'State', 'Make', 'Model']
for col in categorical_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])

print("\nDimensiones del dataset tras el Label encoding:", df.shape)

# Revisamos algunas filas para confirmar el cambio
print("\nDatos después del Label Encoding:")
print(df.head())

# # 2.4 Separación de la variable objetivo y las independientes
# X = df_encoded.drop('Price', axis=1)
# y = df_encoded['Price']

# Separación de la variable objetivo y las independientes
X = df.drop('Price', axis=1)
y = df['Price']

# Escalado de las variables numéricas 'Year' y 'Mileage'
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

   Price  Year  Mileage              City State   Make         Model
0   8995  2014    35725           El Paso    TX  Acura    ILX6-Speed
1  10888  2013    19606  Long Island City    NY  Acura    ILX5-Speed
2   8995  2013    48851           El Paso    TX  Acura    ILX6-Speed
3  10999  2014    39922           Windsor    CO  Acura    ILX5-Speed
4  14799  2016    22142            Lindon    UT  Acura  ILXAutomatic

Valores nulos por columna:
Price      0
Year       0
Mileage    0
City       0
State      0
Make       0
Model      0
dtype: int64

Dimensiones del dataset tras el Label encoding: (852122, 7)

Datos después del Label Encoding:
   Price  Year  Mileage  City  State  Make  Model
0   8995  2014    35725   646     49     1   1194
1  10888  2013    19606  1260     39     1   1193
2   8995  2013    48851   646     49     1   1194
3  10999  2014    39922  2490      6     1   1193
4  14799  2016    22142  1231     50     1   1196


## 3. División del Dataset en Conjuntos de Entrenamiento y Prueba

Separamos los datos en un 80% para entrenamiento y un 20% para prueba. Esto nos permitirá evaluar el rendimiento del modelo en datos no vistos durante el entrenamiento.

In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
print("\nTamaño del conjunto de entrenamiento:", X_train.shape)
print("Tamaño del conjunto de prueba:", X_test.shape)


Tamaño del conjunto de entrenamiento: (681697, 6)
Tamaño del conjunto de prueba: (170425, 6)


## 4. Construcción los modelos de Machine Learning (Regresión)

Se crea un modelo secuencial con dos capas ocultas (64 neuronas cada una) y una capa de salida para regresión (una única neurona). Se compila el modelo utilizando el optimizador Adam, la función de pérdida MSE (error cuadrático medio) y se monitoriza el MAE (error absoluto medio).

In [ ]:
def models(model):
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, y_pred)
    print("Metricas: ")
    print(f"Mae: {mae} , R2_Score: {r2}")
    print("--------------------")

In [8]:
model1 = LinearRegression()
models(model1)

Metricas: 
Mae: 7648.539482853907 , R2_Score: <function r2_score at 0x7fd0bd419ee0>
--------------------


In [ ]:
model = RandomForestRegressor(random_state=42)
model.fit(X_train, y_train)
#Predicciones
y_pred = model.predict(X_test)
#Metricas
print("Metricas del modelo de regresion Random Forest")
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
print(f"MAE: {mae}, RMSE: {rmse}, R²: {r2}")
print("---------------------")

In [ ]:
model = DecisionTreeRegressor(random_state=42)
model.fit(X_train, y_train)
#Predicciones
y_pred = model.predict(X_test)
#Metricas
print("Metricas del modelo de regresion Decision Tree")
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
print(f"MAE: {mae}, RMSE: {rmse}, R²: {r2}")
print("---------------------")

In [ ]:
# Crear y entrenar el modelo con kernel RBF
svr = SVR(kernel='rbf', C=100, epsilon=0.1, gamma='scale')
svr.fit(X_train, y_train)
# Evaluación del modelo
print("Metricas del modelo de regresion SVR")
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
print(f"MAE: {mae}, RMSE: {rmse}, R²: {r2}")
print("---------------------")

Metricas del modelo de regresion
Mean Squared Error:  145763612.32787466
R2 Score:  0.20718790178253965
---------------------


: 

In [ ]:
model = XGBRegressor()
model.fit(X_train, y_train)
#Predicciones
y_pred = model.predict(X_test)
#Metricas
print("Metricas del modelo de regresion XGBRegressor")
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)
print(f"MAE: {mae}, RMSE: {rmse}, R²: {r2}")
print("---------------------")

### 📊 Curva ROC
La curva ROC (Receiver Operating Characteristic) es otra metrica que nos ayuda a determinar como de bien se comporta el modelo distinguiendo entre las clases positivas y negativas y a su vez nos permite seleccionar el mejor modelo para el problema que estamos tratando (Cuanto mas se aproxime a 1 en la grafica mejor desepeño tendra el modelo clasificando los registros).

In [ ]:
from sklearn.metrics import roc_auc_score, roc_curve
models = {
    'LinearRegression': lr,
    'RandomForestRegressor': rf,
    'DecisionTreeRegressor': dt,
    'SuperVectorRegressor': svm,
    'XGBRegressor': xgb
}
plt.figure(figsize=(10, 8))

roc_auc_scores = {}

# Initialize a dictionary to store AUC - ROC scores
roc_auc_scores = {}

# Plot the ROC curves
plt.figure(figsize=(10, 8))

for name, model in models.items():
    # Train the model
    model.fit(X_train, y_train)
    
    # Predict the probabilities
    y_probs = model.predict_proba(X_test)[:, 1]
    
    # Calculate the AUC - ROC score
    roc_auc = roc_auc_score(y_test, y_probs)
    roc_auc_scores[name] = roc_auc
    
    # Compute ROC curve
    fpr, tpr, _ = roc_curve(y_test, y_probs)
    
    # Plot ROC curve
    plt.plot(fpr, tpr, lw=2, label=f'{name} (AUC = {roc_auc:.2f})')

# Plot the diagonal 50% line
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')

# Customize the plot
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve Comparison')
plt.legend(loc="lower right")
plt.show()

# Print the AUC - ROC scores for each model
for name, score in roc_auc_scores.items():
    print(f'{name}: AUC - ROC = {score:.2f}')

## Conclusiones

En este notebook se ha demostrado cómo construir y entrenar un modelo de red neuronal para estimar el precio de vehículos usados. Se han realizado las siguientes tareas:
- **Preprocesamiento de los datos**: limpieza, transformación de variables categóricas y escalado.
- **División en conjuntos de entrenamiento y prueba**.
- **Construcción de un modelo de red neuronal** utilizando Keras.
- **Entrenamiento y visualización de la evolución de las métricas**.
- **Evaluación del modelo** y comparación entre los valores reales y predichos.

Este ejemplo puede ser la base para realizar mejoras adicionales en la arquitectura del modelo, el preprocesamiento de datos o incluso probar otras técnicas de ingeniería de características para mejorar la precisión del modelo.